In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-07-10 22:20:43,-0.6000,73.0833,82.17,70,100,3.20
1,1,Busselton,AU,2022-07-10 22:26:22,-33.6500,115.3333,54.82,63,96,2.80
2,2,Provideniya,RU,2022-07-10 22:26:22,64.3833,-173.3000,51.91,87,75,11.18
3,3,Chapais,CA,2022-07-10 22:26:22,49.7834,-74.8492,60.58,74,97,17.83
4,4,Mildura,AU,2022-07-10 22:14:23,-34.2000,142.1500,44.67,73,100,8.90


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[
    max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-07-10 22:20:43,-0.6000,73.0833,82.17,70,100,3.20
13,13,Vaini,TO,2022-07-10 22:26:25,-21.2000,-175.2000,78.96,78,75,10.36
15,15,Los Zacatones,MX,2022-07-10 22:26:26,22.9500,-102.0500,80.55,25,55,11.92
20,20,Bhinga,IN,2022-07-10 22:26:27,27.7167,81.9333,87.67,59,39,7.74
26,26,Albany,US,2022-07-10 22:25:54,42.6001,-73.9662,81.18,37,67,3.09
27,27,Bandarbeyla,SO,2022-07-10 22:20:29,9.4942,50.8122,75.65,78,56,30.85
29,29,Kaman,IN,2022-07-10 22:26:30,27.6500,77.2667,86.45,70,93,5.21
37,37,Buala,SB,2022-07-10 22:23:08,-8.1450,159.5921,82.54,72,98,5.41
40,40,Mayo,US,2022-07-10 22:26:32,38.8876,-76.5119,83.35,65,75,10.36
41,41,Kongolo,CD,2022-07-10 22:26:32,-5.3833,27.0000,75.27,41,48,3.29


In [9]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [10]:
preferred_cities_df.count()

City_ID       201
City          201
Country       201
Date          201
Lat           201
Lng           201
Max Temp      201
Humidity      201
Cloudiness    201
Wind Speed    201
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.17,-0.6000,73.0833,
13,Vaini,TO,78.96,-21.2000,-175.2000,
15,Los Zacatones,MX,80.55,22.9500,-102.0500,
20,Bhinga,IN,87.67,27.7167,81.9333,
26,Albany,US,81.18,42.6001,-73.9662,
27,Bandarbeyla,SO,75.65,9.4942,50.8122,
29,Kaman,IN,86.45,27.6500,77.2667,
37,Buala,SB,82.54,-8.1450,159.5921,
40,Mayo,US,83.35,38.8876,-76.5119,
41,Kongolo,CD,75.27,-5.3833,27.0000,


In [12]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [13]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'AeJbb3dYQH1j1ZuQL6KcxCNs8cPejBT8oZYoa5qlyHaibhoTBFaap7lPo_VQ402bUHAkRJC7G5NxUMux8NtP-6QhgMjsUJAoZ8-c_zqjX9OT5JWtMv286Mw4KoikwNHvvc6sJk06NBeYliqjX7-ILrGTPV4yo0M_zTmby_6oWohn8b-edVhOCW2hPHOTNbcS7-3VXlMYEqdRBUH8UVlohClnUM9BiT6WcvVo6Dv8NIGX1EZh32rMjlVl-1WHh1Aw7cg9e1c06-_OfPg1wECQQz-2XiXCdcnEAKxU92WL6zIN16CtNlEjy8RJ7LoTteqGD1RVudBPrU7TcLMwEPENZpGVC9n1VRObP_QW9_KXuR5a23APH5S3HYqFrzBOgJSzR9R2qcqZ6NqTeIy7lmqcQdP9ON-SgaqoC7fdqjMiNobzu-DGs9PMXgmyzx5N',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.17,-0.6000,73.0833,Scoop Guest House
13,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
15,Los Zacatones,MX,80.55,22.9500,-102.0500,
20,Bhinga,IN,87.67,27.7167,81.9333,ANAND MISHTAN BHANDAR AND COFFEE TEA SETAL
26,Albany,US,81.18,42.6001,-73.9662,
27,Bandarbeyla,SO,75.65,9.4942,50.8122,JABIR HOTEL
29,Kaman,IN,86.45,27.6500,77.2667,Dr.kishan kaman
37,Buala,SB,82.54,-8.1450,159.5921,Maringe Lagoon Lodge
40,Mayo,US,83.35,38.8876,-76.5119,Resorts Resource Group International
41,Kongolo,CD,75.27,-5.3833,27.0000,GUEST HOUSE OASIS


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))